In [35]:
# ### get link products
# //*[@id="root-app"]/div/div/section/ol/li/div/div/a/@href
# #### Product name
# //*[@id="root-app"]/div/div/div/div/div/div/div/div/div/div/h1/text()
# #### get link reviews
# //*[@id="root-app"]/div/div/div/div/div/div/div/div/section/a/@href
# ##### Stars
# //*[@id="reviews-capability.desktop"]/section/div/div/div/div/div/article/div/label/text()
# ##### review
# //*[@id="reviews-capability.desktop"]/section/div/div/div/div/div/article/p/text()
# ### get next page link
# //*[@id="root-app"]/div/div/section/div/ul/li/a/@href

In [36]:
import re
import json
import pandas as pd
import requests
import lxml.html as html

### Scrap products from categories

In [52]:

category_url = "https://lista.mercadolivre.com.br/casa-moveis-decoracao/"
try:
    all_links = []
    while category_url:
        response = requests.get(category_url)
        if response.status_code == 200:
            content = response.content.decode('utf-8')
            parsed = html.fromstring(content)
            links_products = parsed.xpath('//*[@id="root-app"]/div/div/section/ol/li/div/div/a/@href')
            all_links += links_products
            next_page = parsed.xpath('//li[@class="andes-pagination__button andes-pagination__button--next"]/a/@href')
            print(next_page)
            category_url = next_page[0]

except Exception as e:
    print(e)

['https://lista.mercadolivre.com.br/casa-moveis-decoracao/_Desde_49_NoIndex_True']
['https://lista.mercadolivre.com.br/casa-moveis-decoracao/_Desde_97_NoIndex_True']
['https://lista.mercadolivre.com.br/casa-moveis-decoracao/_Desde_145_NoIndex_True']
['https://lista.mercadolivre.com.br/casa-moveis-decoracao/_Desde_193_NoIndex_True']
['https://lista.mercadolivre.com.br/casa-moveis-decoracao/_Desde_241_NoIndex_True']
['https://lista.mercadolivre.com.br/casa-moveis-decoracao/_Desde_289_NoIndex_True']
['https://lista.mercadolivre.com.br/casa-moveis-decoracao/_Desde_337_NoIndex_True']
['https://lista.mercadolivre.com.br/casa-moveis-decoracao/_Desde_385_NoIndex_True']
['https://lista.mercadolivre.com.br/casa-moveis-decoracao/_Desde_433_NoIndex_True']
['https://lista.mercadolivre.com.br/casa-moveis-decoracao/_Desde_481_NoIndex_True']
['https://lista.mercadolivre.com.br/casa-moveis-decoracao/_Desde_529_NoIndex_True']
['https://lista.mercadolivre.com.br/casa-moveis-decoracao/_Desde_577_NoIndex_T

In [53]:
len(all_links)

2113

### get product reviews 

In [54]:
def get_reviews(request) -> json:
    
    product_content = request.content.decode('utf-8')
    product_parsed = html.fromstring(product_content)
    reviews = product_parsed.xpath('/html/body/p/text()')
    reviews_json = json.loads(reviews[0])
    
    return reviews_json

In [42]:
page_reviews = 'https://www.mercadolivre.com.br/noindex/catalog/reviews/MLB{}/scroll?siteId=MLB&type=all&isItem=false&offset={}&limit=1'

product_id = product_id = re.search("/MLB-?(\d{4,})", all_links[1])

product_reviews = page_reviews.format(product_id.group(1), 0)
product_request = requests.get(product_reviews)
print(product_request.status_code)
product_content = product_request.content.decode('utf-8')
product_parsed = html.fromstring(product_content)
reviews = product_parsed.xpath('/html/body/p/text()')
reviews_json = json.loads(reviews[0])
print(reviews_json)


200
{'reviews': [{'id': 92464570, 'type': 'REVIEW', 'rating': 5, 'title': {'text': 'ótimo produto com preço bom', 'class_name': 'ui-review-view__comments__review-comment__title'}, 'comment': {'content': {'text': 'Omo né !!! despensa comentários. \nMais barato que nos mercados do meu bairro e eu amo é o unico que deixa as roupas branquinhas, fora o omo so utilizo ariel liquido. Combinação matadora de sujeira.', 'class_name': 'ui-review-view__comments__review-comment', 'see_more': 'Ver mais'}, 'time': {'text': 'Mais de 1 ano', 'class_name': 'ui-review-view__comments__review-comment__date'}, 'denounce': {'text': 'Denunciar', 'url': '/noindex/denounce/?item_id=MLB1397481414&element_id=92464570&element_type=REV', 'track': {'event_data': {'catalog_product_id': 'MLB17001048', 'review_id': 92464570, 'type': 'denounce'}, 'path': '/reviews/actions'}, 'class_name': 'ui-review-view__comments__review-comment-denunce'}}, 'actions': [{'id': 'LIKE', 'type': 'LIKE', 'value': 7, 'track': {'event_data': 

In [45]:
title = reviews_json["reviews"][0]["title"]["text"]
coments = reviews_json["reviews"][0]["comment"]["content"]["text"]
rating = reviews_json["reviews"][0]["rating"]

print(f'''
Rating:  {rating}\n
Title:   {title}\n
Comment: \n{coments}
        ''')


Rating:  5

Title:   ótimo produto com preço bom

Comment: 
Omo né !!! despensa comentários. 
Mais barato que nos mercados do meu bairro e eu amo é o unico que deixa as roupas branquinhas, fora o omo so utilizo ariel liquido. Combinação matadora de sujeira.
        


### Extract reviews

In [50]:
def scrap_reviews(url_product):
    
    page_reviews = 'https://www.mercadolivre.com.br/noindex/catalog/reviews/MLB{}/scroll?siteId=MLB&type=all&isItem=true&offset={}&limit=1'
    altern_page_reviews = 'https://www.mercadolivre.com.br/noindex/catalog/reviews/MLB{}/scroll?siteId=MLB&type=all&isItem=false&offset={}&limit=1'
    
    try:
        counter = 0
        product = url_product
        product_id = re.search("/MLB-?(\d{4,})", product)
        while True:
            
            product_reviews = page_reviews.format(product_id.group(1), counter)
            product_request = requests.get(product_reviews)

            if product_request.status_code != 200:
                product_reviews = altern_page_reviews.format(product_id.group(1), counter)
                product_request = requests.get(product_reviews)
                reviews = get_reviews(product_request)
                print(reviews)
            
                if len(reviews["reviews"][0]) == 1:
                    break
                else:
                    return reviews

            else:
                product_request = requests.get(product_reviews)
                reviews = get_reviews(product_request)
                print(reviews)
                counter += 1
                print(counter)

                if len(reviews["reviews"][0]) == 1:
                    break
                else:
                    return reviews
            
        counter += 1
        print(counter)
        
    except:
        next
        

In [4]:
page_reviews = 'https://www.mercadolivre.com.br/noindex/catalog/reviews/MLB{}/scroll?siteId=MLB&type=all&isItem=true&offset={}&limit=1'
altern_page_reviews = 'https://www.mercadolivre.com.br/noindex/catalog/reviews/MLB{}/scroll?siteId=MLB&type=all&isItem=false&offset={}&limit=1'

In [56]:
try:
    counter = 0
    product = all_links[0]
    product_id = re.search("/MLB-?(\d{4,})", all_links[0])
    while True:
        
        product_reviews = page_reviews.format(product_id.group(1), counter)
        product_request = requests.get(product_reviews)

        if product_request.status_code != 200:
            product_reviews = altern_page_reviews.format(product_id.group(1), counter)
            product_request = requests.get(product_reviews)
            reviews = get_reviews(product_request)
            print(reviews)
        
            if len(reviews["reviews"][0]) == 1:
                break

        else:
            product_request = requests.get(product_reviews)
            reviews = get_reviews(product_request)
            print(reviews)

            if len(reviews["reviews"][0]) == 1:
                break
        
        counter += 1
        print(counter)
        
except:
    next


{'reviews': [{'id': 97458121, 'type': 'REVIEW', 'rating': 3, 'title': {'text': 'bom ', 'class_name': 'ui-review-view__comments__review-comment__title'}, 'comment': {'content': {'text': 'Achei bom detergente ipê mas o valor tá um pouco a cima.', 'class_name': 'ui-review-view__comments__review-comment', 'see_more': 'Ver mais'}, 'time': {'text': 'Mais de 1 ano', 'class_name': 'ui-review-view__comments__review-comment__date'}, 'denounce': {'text': 'Denunciar', 'url': '/noindex/denounce/?item_id=MLB1484186426&element_id=97458121&element_type=REV', 'track': {'event_data': {'catalog_product_id': 'MLB18392113', 'review_id': 97458121, 'type': 'denounce'}, 'path': '/reviews/actions'}, 'class_name': 'ui-review-view__comments__review-comment-denunce'}}, 'actions': [{'id': 'LIKE', 'type': 'LIKE', 'value': 2, 'track': {'event_data': {'catalog_product_id': 'MLB18392113', 'review_id': 97458121, 'type': 'like'}, 'path': '/reviews/actions'}, 'icon': {'empty_icon': 'ThumbUpOutlined', 'full_icon': 'ThumbU

### Simple Scraper

In [17]:
product = requests.get(all_links[1])
product = product.content.decode('utf-8')
product_parsed = html.fromstring(product)

product_name = product_parsed.xpath('//*[@id="root-app"]/div/div/div/div/div/div/div/div/div/div/h1/text()')
review_links = product_parsed.xpath('//*[@id="root-app"]/div/div/div/div/div/div/div/div/section/a/@href')

In [21]:
review_links

['/noindex/catalog/reviews/MLB16084355?noIndex=true&access=view_all&modal=true&controlled=true']

In [23]:
review_page = 'http://mercadolivre.com.br' + review_links[0]
reviews_list = requests.get(review_page)
reviews_decode = reviews_list.content.decode('utf-8')
reviews_parsed = html.fromstring(reviews_decode)

reviews = reviews_parsed.xpath('//*[@id="reviews-capability.desktop"]/section/div/div/div/div/div/article/p/text()')
stars = reviews_parsed.xpath('//*[@id="reviews-capability.desktop"]/section/div/div/div/div/div/article/div/label/text()')

In [26]:
print(review_page)
print(reviews)
print(stars)

http://mercadolivre.com.br/noindex/catalog/reviews/MLB16084355?noIndex=true&access=view_all&modal=true&controlled=true
['Prático e macio. Sou deficiente física e tive que instalar um dispensar de inox no meu banheiro para usar melhor o sanitário. Meu braço esquerdo está paralisado após um avc.', 'Não tem muito oque falar, quem conhece sabe que neve é o melhor.', 'Amei, ótima qualidade e super macio. Além do fato do custo e benefício valer muito a pena.', 'Bom produto. Já sou consumidor deste produto há um bom tempo.', 'Sempre usei neve, mas é tão bom poder receber em casa, a um preço bom, e me sentir segura e bem atendida! obrigada.']
['5', '5', '5', '4', '5']


In [25]:
len(reviews)

5